# Machine Problem  
Nathaniel Cruel and Renzo Virtucio  
Department of Computer Science, University of the Philippines Diliman  
CS 180: Artificial Intelligence  
Professor Carlo Raquel  
June 15, 2021
***

## Introduction to the Problem
## Rationale
## Objective

## Methodology

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm

students_math = pd.read_csv('student/student-mat.csv', delimiter=';', header=0)
display(students_math)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,...,5,5,4,4,5,4,11,9,9,9
391,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,3,14,16,16
392,MS,M,21,R,GT3,T,1,1,other,other,...,5,5,3,3,3,3,3,10,8,7
393,MS,M,18,R,LE3,T,3,2,services,other,...,4,4,1,3,4,5,0,11,12,10


In [2]:
## PREPROCESSING ##
# Encoding binary attributes
students_math.school = students_math.school.map({"GP":0, "MS":1})
students_math.sex = students_math.sex.map({"F":0, "M":1})
students_math.address = students_math.address.map({"U":0, "R":1})
students_math.famsize = students_math.famsize.map({"LE3":0, "GT3":1})
students_math.Pstatus = students_math.Pstatus.map({"T":0, "A":1})
students_math.schoolsup = students_math.schoolsup.map({"yes":0, "no":1})
students_math.famsup = students_math.famsup.map({"yes":0, "no":1})
students_math.paid = students_math.paid.map({"yes":0, "no":1})
students_math.activities = students_math.activities.map({"yes":0, "no":1})
students_math.nursery = students_math.nursery.map({"yes":0, "no":1})
students_math.higher = students_math.higher.map({"yes":0, "no":1})
students_math.internet = students_math.internet.map({"yes":0, "no":1})
students_math.romantic = students_math.romantic.map({"yes":0, "no":1})

# Encoding nominal attributes
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

job_types = ["teacher", "health", "services", "at_home", "other"]
reason_types = ["home", "reputation", "course", "other"]
guardian_types = ["mother", "father", "other"]
Mjob_dum = pd.get_dummies(students_math['Mjob'], prefix='Mjob')
Fjob_dum = pd.get_dummies(students_math['Fjob'], prefix='Fjob')
reason_dum = pd.get_dummies(students_math['reason'], prefix='reason')
guardian_dum = pd.get_dummies(students_math['guardian'], prefix='guardian')

students_math = pd.concat([students_math, Mjob_dum, Fjob_dum, reason_dum, guardian_dum], axis=1)
students_math.drop(['Mjob', 'Fjob', 'reason', 'guardian'], axis=1, inplace=True)
students_math

# Add a column describing if G3 (final grade) is high or low
# High = 12-20 (60% of 20 and above)
# Low = 0-11 (below 60% of 20)

students_math["quality"] = (students_math.G3 >= 12).map({True: 1, False: 0})
display(students_math)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,traveltime,studytime,...,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_other,reason_reputation,guardian_father,guardian_mother,guardian_other,quality
0,0,0,18,0,1,1,4,4,2,2,...,0,1,1,0,0,0,0,1,0,0
1,0,0,17,0,1,0,1,1,1,2,...,0,0,1,0,0,0,1,0,0,0
2,0,0,15,0,0,0,1,1,1,2,...,0,0,0,0,1,0,0,1,0,0
3,0,0,15,0,1,0,4,2,1,3,...,1,0,0,1,0,0,0,1,0,1
4,0,0,16,0,1,0,3,3,1,2,...,0,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,1,1,20,0,0,1,2,2,1,2,...,1,0,1,0,0,0,0,0,1,0
391,1,1,17,0,0,0,3,1,2,1,...,1,0,1,0,0,0,0,1,0,1
392,1,1,21,1,1,0,1,1,1,1,...,0,0,1,0,0,0,0,0,1,0
393,1,1,18,1,0,0,3,2,3,1,...,0,0,1,0,0,0,0,1,0,0


In [3]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn import svm
import statsmodels.api as sm

y = students_math['quality']
X = students_math.drop(['quality'], axis=1)
X.shape, y.shape

def calculate_vif(data):
    vif_df = pd.DataFrame(columns = ['Var', 'Vif'])
    x_var_names = data.columns
    for i in range(0, x_var_names.shape[0]):
        y = data[x_var_names[i]]
        x = data[x_var_names.drop([x_var_names[i]])]
        r_squared = sm.OLS(y,x).fit().rsquared
        vif = round(1/(1-r_squared),2)
        vif_df.loc[i] = [x_var_names[i], vif]
    return vif_df.sort_values(by = 'Vif', axis = 0, ascending=False, inplace=False)

X = X.drop(['guardian_other', 'Fjob_other', 'reason_course', 'Mjob_other', 'age', 
            'G2', 'G1', 'Medu', 'famrel', 'freetime', 'goout', 'Fedu', 'Walc', 
            'schoolsup', 'health', 'G3', 'studytime', 'traveltime'], axis=1)
calculate_vif(X)

,Var,Vif
13,Dalc,4.36
3,famsize,3.36
12,romantic,3.23
26,guardian_father,2.96
7,paid,2.71
1,sex,2.45
8,activities,2.10
6,famsup,1.95
17,Mjob_services,1.90
21,Fjob_services,1.81


In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=15)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

model = svm.SVC()
model.fit(X_train, y_train)

from sklearn.metrics import classification_report
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))
print(model.score(X_test, y_test))

(237, 28) (237,)
(158, 28) (158,)
              precision    recall  f1-score   support

           0       0.60      1.00      0.75        95
           1       0.00      0.00      0.00        63

    accuracy                           0.60       158
   macro avg       0.30      0.50      0.38       158
weighted avg       0.36      0.60      0.45       158

0.6012658227848101


C:\anacondaa\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [5]:
from sklearn.model_selection import GridSearchCV
  
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ('linear', 'rbf', 'poly', 'sigmoid')} 
  
grid = GridSearchCV(svm.SVC(), param_grid, refit = True, verbose = 3, n_jobs=-1)
  
# fitting the model for grid search
grid.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 393 tasks      | elapsed:   10.1s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.1min finished


GridSearchCV(estimator=SVC(), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ('linear', 'rbf', 'poly', 'sigmoid')},
             verbose=3)

In [6]:
# print best parameter after tuning
print(grid.best_params_)
  
# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

{'C': 1000, 'gamma': 0.0001, 'kernel': 'sigmoid'}
SVC(C=1000, gamma=0.0001, kernel='sigmoid')


In [7]:
grid_predictions = grid.predict(X_test)
  
# print classification report
print(classification_report(y_test, grid_predictions))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, grid_predictions))

              precision    recall  f1-score   support

           0       0.70      0.73      0.71        95
           1       0.56      0.52      0.54        63

    accuracy                           0.65       158
   macro avg       0.63      0.63      0.63       158
weighted avg       0.64      0.65      0.64       158

[[69 26]
 [30 33]]


## Data and Analysis

## Conclusion

## Individual Contributions